In [42]:
from langchain_community.document_loaders import UnstructuredFileLoader
from nltk.tokenize import sent_tokenize
import PyPDF2

def extract_text_from_pdf(pdf_path):
    # Use PyPDF2 to extract text from the PDF
    pdf_file = open(pdf_path, 'rb')
    pdf_reader = PyPDF2.PdfReader(pdf_file)
    text = ''
    # Loop through each page of the PDF
    for page_number in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_number]
        text += page.extract_text()
    pdf_file.close()
    return text

def chunk_text(text):
    #breaking long text into sentences
    sentences = sent_tokenize(text)
    #emppty list to store the chunks of each in
    chunks = []
    current_chunk = ""
    #for each sentence 
    for sentence in sentences:
        #to not pass 500 in sentences len for each chunk
        if len(current_chunk) + len(sentence) < 500:  
            current_chunk += " " + sentence
        #if it excedd 500 then we pass to the next chunk & append the previous
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence
    #if there s any left sentences we append them
    if current_chunk:
        chunks.append(current_chunk.strip())
    return chunks

#saving to a file 
def save_chunks_to_file(text, output_file):
    chunks = chunk_text(text)  # Generate chunks
    with open(output_file, 'w', encoding='utf-8') as f:
        for chunk in chunks:
            f.write(chunk + '\n')

pdf_path = 'infos/pdf2.pdf'
pdf_text = extract_text_from_pdf(pdf_path)

loader = UnstructuredFileLoader(pdf_text)

processed_text = loader

# Print the processed text
print(processed_text)

output_file = "chunked.txt"
save_chunks_to_file(pdf_text, output_file)
